In [5]:
!pip install flair

In [13]:
!python -m spacy download ru_core_news_sm

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 15.3/15.3 MB 7.9 MB/s eta 0:00:00a 0:00:01
  Using cached pymorphy3-2.0.2-py3-none-any.whl.metadata (1.8 kB)
  Using cached DAWG_Python-0.7.2-py2.py3-none-any.whl.metadata (7.0 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.4/8.4 MB 7.4 MB/s eta 0:00:0000:0100:01
✔ Download and installation successful
You can now load the package via spacy.load('ru_core_news_sm')


In [33]:
!pip install 'transformers[torch]'

  Using cached transformers-4.47.0-py3-none-any.whl.metadata (43 kB)
  Using cached tokenizers-0.21.0-cp39-abi3-macosx_11_0_arm64.whl.metadata (6.7 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.6/2.6 MB 380.7 kB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.1/10.1 MB 1.3 MB/s eta 0:00:0000:010:010m
  Attempting uninstall: tokenizers
    Found existing installation: tokenizers 0.20.3
    Uninstalling tokenizers-0.20.3:
      Successfully uninstalled tokenizers-0.20.3


In [19]:
import re
import numpy as np
import pandas as pd
import joblib

from tqdm import tqdm

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report
from flair.models import SequenceTagger
from flair.data import Sentence

from transformers import MarianMTModel, MarianTokenizer

In [40]:
genres_titles = [
  "Action",
  "Adventure",
  "Animation",
  "Biography",
  "Comedy",
  "Crime",
  "Documentary",
  "Drama",
  "Family",
  "Fantasy",
  "Game-Show",
  "History",
  "Horror",
  "Music",
  "Musical",
  "Mystery",
  "News",
  "Reality-TV",
  "Romance",
  "Sci-Fi",
  "Short",
  "Sport",
  "Talk-Show",
  "Thriller",
  "War",
  "Western"
]

In [41]:
films_titles = [
  "'Star Trek: Deep Space Nine': Behind the Scenes",
  "'Star Wars': Feel the Force",
  "1983 MLB All-Star Game",
  "1987 NBA All-Star Game",
  "1997 MLB All-Star Game",
  "1998 MLB All-Star Game",
  "1999 MLB All-Star Game",
  "2-Star",
  "2000 MLB All-Star Game",
  "2001 MLB All-Star Game",
  "2002 MLB All-Star Game",
  "2003 MLB All-Star Game",
  "2004 MLB All-Star Game",
  "2005 MLB All-Star Game",
  "2006 MLB All-Star Game",
  "2007 MLB All-Star Game",
  "2008 MLB All-Star Game",
  "2011 NBA All-Star Game",
  "5 Star Day",
  "50 Years of Star Trek",
  "7 Star Grand Mantis",
  "9 Muses of Star Empire",
  "9 Star Hotel",
  "9-1-1: Lone Star",
  "A 5 Star",
  "A Beautiful Star",
  "A Christmas Star",
  "A Few Moments with Eddie Cantor, Star of 'Kid Boots'",
  "A Five Star Life",
  "A Football Star Called Divine: Life and Work of Ademir da Guia",
  "A Gold Star Kid",
  "A Man from This Star",
  "A Mock Time: A Star Trek Wedding",
  "A Movie Star",
  "A Star Athlete",
  "A Star Is Bored",
  "A Star Is Born",
  "A Star Is Born",
  "A Star Is Born",
  "A Star Is Born",
  "A Star Is Born World Premiere",
  "A Star Is Falling Upwards",
  "A Star Is Hatched",
  "A Star Named Ayrton Senna",
  "A Star and Two Coffees",
  "A Star for Christmas",
  "A Star for Two",
  "A Star is Born",
  "A Voyage Around a Star",
  "AWA All-Star Wrestling",
  "Adventures in Odyssey: Star Quest",
  "Afghan Star",
  "Afghan Star",
  "Alex Jones Explains the Star Wars Prequels",
  "All About Ann: Governor Richards of the Lone Star State",
  "All Star Comedy Jam",
  "All Star Comedy Jam: Live from South Beach",
  "All Star Family Fortunes",
  "All Star Mr & Mrs",
  "All Star Revue",
  "All-Star 25th Birthday: Stars and Street Forever!",
  "All-Star Academy",
  "All-Star American Destiny Trek",
  "All-Star Birthday Party at Annapolis",
  "All-Star Comedy Birthday Party from West Point",
  "All-Star Family Feud Special",
  "All-Star Party for 'Dutch' Reagan",
  "All-Star Party for Carol Burnett",
  "All-Star Party for Lucille Ball",
  "All-Star Superman",
  "All-Star Vaudeville",
  "Amateur Porn Star Killer",
  "Amateur Porn Star Killer 2",
  "Amateur Porn Star Killer 3: The Final Chapter",
  "America's All-Star Tribute to Elizabeth Taylor",
  "America's Five Star Heroes: Gods of War",
  "American Beauty Star",
  "Amy Winehouse: Fallen Star",
  "An All-Star Celebration Honoring Martin Luther King Jr.",
  "An All-Star Toast to the Improv",
  "An All-Star Tribute to Brian Wilson",
  "An All-Star Tribute to Elizabeth Taylor",
  "An All-Star Tribute to John Wayne",
  "An All-Star Tribute to Johnny Cash",
  "An All-Star Tribute to Joni Mitchell",
  "Angry Birds Star Wars",
  "Angry Birds Star Wars Cinematic Trailer",
  "Angry Birds Star Wars II",
  "Antonio Carlos Jobim: An All-Star Tribute",
  "Anything for a Pop Star",
  "Atlantis: Secret Star Mappers of a Lost World",
  "Axl Rose: The Prettiest Star",
  "Bad Girls All Star Battle",
  "Barbie: Star Light Adventure",
  "Barney's Christmas Star",
  "Battle Star Wars",
  "Be My Star",
  "Beam Me Up - Die große Star Trek Show",
  "Bette Davis - Star und Rebellin",
  "Bewitched: Star Shots",
  "Beyond the Farthest Star",
  "Big Star's Little Star",
  "Big Star: Nothing Can Hurt Me",
  "Billion Star Hotel",
  "Billy Star",
  "Black Star and the Golden Bat",
  "Bob Hope: Hollywood's Brightest Star",
  "Boban the Hockey Star",
  "Boku to Star no 99 nichi",
  "Bright Star",
  "Brightest Star",
  "Bring Back... Star Trek",
  "Bring Back... Star Wars",
  "Britney Spears: 'Star Baby' Scrapbook",
  "Broken Star",
  "Bucketheads: A Star Wars Story",
  "Bucky Larson: Born to Be a Star",
  "Building Star Trek",
  "Buzz Lightyear of Star Command",
  "Buzz Lightyear of Star Command",
  "Buzz Lightyear of Star Command: The Adventure Begins",
  "Bye Bye Star-Club",
  "Captain Star",
  "Carrie Underwood: An All-Star Holiday Special",
  "Catch a Christmas Star",
  "Catch a Falling Star",
  "Chasing the Star",
  "Chatur Singh Two Star",
  "Child Star Confidential",
  "Child Star Psychologist",
  "Child Star: The Shirley Temple Story",
  "Children of the Dog Star",
  "Chris Hardwick's All-Star Celebrity Bowling",
  "Christmas Star",
  "Cloud Capped Star",
  "Comedy Central's All-Star Non-Denominational Christmas Special",
  "Confessions of an Action Star",
  "Confessions of an Action Star",
  "Cory in the House: All Star Edition",
  "Crescent Star",
  "DD Fist of the North Star",
  "Dance Star",
  "Dance with Death: The Ufa-Star Sybille Schmitz",
  "Dancing Star",
  "Dark Star",
  "Dark Star: HR Gigers Welt",
  "Date My Porn Star",
  "Daytime Shooting Star",
  "Death Star Repairmen",
  "Deep Inside Clint Star",
  "Demolishing and Building Up the Star Theatre",
  "Diary of an Ex-Child Star",
  "Dick Ho: Asian Male Porn Star",
  "Dickie Roberts: Former Child Star",
  "Diego Star",
  "Disney Channel New Year Star Showdown",
  "Disney Star Darlings",
  "Dog Star",
  "Dog Star Man",
  "Dog Star Man: Part I",
  "Dog Star Man: Part II",
  "Dog Star Man: Part III",
  "Dog Star Man: Part IV",
  "Double Digits: The Story of a Neighborhood Movie Star",
  "Dr. T, Lone Star Vet",
  "Earth Star Voyager",
  "Ed Sullivan All-Star Comedy Special",
  "Elvis All-Star Tribute",
  "Empire of Dreams: The Story of the 'Star Wars' Trilogy",
  "Evening Star",
  "Exclusiv - Das Star-Magazin",
  "Exile: A Star Wars Story",
  "Fall in Love Like a Star",
  "Falling Star",
  "Fallout: Red Star",
  "Fashion Star",
  "Fashion Star",
  "Film Star",
  "Fist of the North Star",
  "Fist of the North Star",
  "Fist of the North Star",
  "Fist of the North Star 2",
  "Fist of the North Star: New Saviour Legend",
  "Fist of the North Star: The Legend of Yuria",
  "Fist of the North Star: The Shin Saga",
  "Fist of the North Star: The Souther Saga",
  "Fist of the North Star: The Toki Saga",
  "Five Star",
  "Five Star",
  "Five Star Existence",
  "Five Star Final",
  "Five Star Fouad",
  "Five Star Hotel",
  "Five Star Stories",
  "Flaming Star",
  "Fluffy Cumsalot, Porn Star",
  "Follow a Star",
  "Food Network Star",
  "Food Network Star Kids",
  "Football's Next Star",
  "Ford Star Jubilee",
  "Four Star Playhouse",
  "Frat Star",
  "From 'Star Wars' to 'Jedi': The Making of a Saga",
  "From Star Wars to Star Wars: The Story of Industrial Light & Magic",
  "Fullmetal Alchemist: The Sacred Star of Milos",
  "Futari wa purikyua: Splash Star",
  "Garrincha: Lonely Star",
  "Gene Tierney a Forgotten Star",
  "General Electric's All-Star Anniversary",
  "George and the Christmas Star",
  "Glow Up: Britain's Next Make-Up Star",
  "Goal Star",
  "Gold Star",
  "Grand Star",
  "Greta Garbo: A Lone Star",
  "Grown Up Movie Star",
  "HGTV Design Star",
  "Han: A Star Wars Adventure",
  "Hannah Montana: Learn to Be a Pop Star",
  "Hayley Wagner, Star",
  "He Found a Star",
  "Hebe: The Brazilian Star",
  "Hedy Lamarr: Secrets of a Hollywood Star",
  "Here, Beneath the North Star",
  "Hibi Rock: Puke Afro and the Pop Star",
  "Highway Star",
  "Holly Star",
  "Hour of the Star",
  "Howie Mandel All-Star Comedy Gala",
  "Hugo: The Movie Star",
  "Hyper Sapien: People from Another Star",
  "I Am Not a Rock Star",
  "I Am a Star",
  "I Know My Kid's a Star",
  "I Wanna Be a Porn Star!",
  "I Wanna Be a Soap Star",
  "I Was a Network Star",
  "I'm a Porn Star",
  "I've Never Seen Star Wars",
  "IMDb's 20th Anniversary Star of the Day",
  "Ich bin ein Star - Holt mich hier raus!",
  "In Concert Cat Stevens: Moon & Star",
  "Inside Star Trek: The Real Story",
  "Instant Star",
  "Inter Star Wars: The Awaking Force",
  "Invasion of the Star Creatures",
  "JJ Star... How Embarrassing",
  "Jade: The Reality Star Who Changed Britain",
  "Jason of Star Command",
  "Jenna's American Sex Star",
  "Jimmy Kimmel Live's All-Star Salute to Jimmy Kimmel Live!",
  "Jingle & Bell's Christmas Star",
  "Joan Crawford: Always the Star",
  "Joan Crawford: The Ultimate Movie Star",
  "John Wycliffe: The Morning Star",
  "Journey to the Christmas Star",
  "Journey's End: The Saga of Star Trek - The Next Generation",
  "Justin Bieber: A Star Was Born",
  "Kaleido Star",
  "Kamen Rider Super Den-O Trilogy: Episode Red - Zero's Star Twinkle",
  "Kara: A Star Wars Story",
  "Keiko the Untold Story of the Star of Free Willy",
  "Kenobi: A Star Wars Story",
  "Kidz Bop: Everyone's a Star!",
  "Kin: Fallen Star",
  "Kinect Star Wars",
  "Kinect Star Wars: Duel",
  "King Star King",
  "Kirby Star Allies",
  "Kirby Super Star",
  "Kitakyushu Stories Chapter One: Kimi the Star",
  "Kre-O Star Trek",
  "LEGO Star Wars: The Resistance Rises",
  "Larry the Cable Guy's Star-Studded Christmas Extravaganza",
  "Laura's Christmas Star",
  "Laura's Star",
  "Laura's Star and the Dream Monster",
  "Laura's Star and the Mysterious Dragon Nian",
  "Leaked Disney Star Wars 7 Trailer",
  "Legends of the Dark King: A Fist of the North Star Story",
  "Lego Star Wars II: The Original Trilogy",
  "Lego Star Wars III: The Clone Wars",
  "Lego Star Wars: All-Stars",
  "Lego Star Wars: Bombad Bounty",
  "Lego Star Wars: Droid Tales",
  "Lego Star Wars: Revenge of the Brick",
  "Lego Star Wars: The Complete Saga",
  "Lego Star Wars: The Empire Strikes Out",
  "Lego Star Wars: The Force Awakens",
  "Lego Star Wars: The Freemaker Adventures",
  "Lego Star Wars: The Padawan Menace",
  "Lego Star Wars: The Quest for R2-D2",
  "Lego Star Wars: The Video Game",
  "Lego Star Wars: The Yoda Chronicles",
  "Lego Star Wars: The Yoda Chronicles - Attack of the Jedi",
  "Lego Star Wars: The Yoda Chronicles - Menace of the Sith",
  "Lego Star Wars: The Yoda Chronicles - Secret Plans Are Revealed",
  "Lego Star Wars: The Yoda Chronicles - The Dark Side Rises",
  "Lego Star Wars: The Yoda Chronicles - The Phantom Clone",
  "Lego Star Wars: The Yoda Chronicles - Who Let the Clones Out",
  "Leonard Nimoy: Star Trek Memories",
  "Let There Be Light: The Odyssey of Dark Star",
  "Lick the Star",
  "Like a Shooting Star",
  "Like a Star Shining in the Night",
  "Lone Star",
  "Lone Star",
  "Lone Star",
  "Lone Star Justice",
  "Lone Star Law",
  "Lone Star Law Men",
  "Lone Star Raiders",
  "Lone Star Restoration",
  "Lone Star State of Mind",
  "Lone Star Struck",
  "Look for a Star",
  "Lucky Star",
  "Lucky Star",
  "Lucky Star",
  "Lucky Star",
  "Lucky Star",
  "Lunar: The Silver Star",
  "Magic Star Magical Emi",
  "Make Me a Star",
  "Man Without a Star",
  "Mazzy Star: Fade Into You, Color Version",
  "Meet Your Star",
  "Mega Man: Upon a Star",
  "Mega Star",
  "MegaMan Star Force",
  "Mercedes 'Lucky Star'",
  "Michael Jackson - Mega Star",
  "Michael Stahl-David: Behind the Star",
  "Mission Star",
  "Molly, Star-Racer",
  "Monopoly Star Wars",
  "Morning Star",
  "Movie Star, American Style or; LSD, I Hate You",
  "Mr Black: Green Star",
  "Mr. Hollywood Star",
  "Mr. Plinkett's Star Trek 2009 Review",
  "Mr. Plinkett's The Star Wars Awakens Review",
  "My Dad the Rock Star",
  "My Guide to Becoming a Rock Star",
  "My Love from Another Star",
  "My Love from the Star",
  "My Lucky Star",
  "My Lucky Star",
  "My Lucky Star",
  "Nashville Star",
  "Natural Born Star",
  "New Fist of the North Star",
  "Next Action Star",
  "Next Big Star",
  "North Star",
  "North Star",
  "North from the Lone Star",
  "Northern Star",
  "Nouvelle Star",
  "Oban Star-Racers",
  "Ocean Star",
  "Odyssey: A Star Wars Story",
  "One Love: The Bob Marley All-Star Tribute",
  "One Night One Star: Usher Live",
  "Our Star Trek: The Fifty Year Mission",
  "Outlaw Star",
  "Overwatch: Shooting Star",
  "P-Star Rising",
  "Paper Mario: Sticker Star",
  "Paper Star",
  "Perry Mason: The Case of the All-Star Assassin",
  "Perry Mason: The Case of the Shooting Star",
  "Pet Star",
  "Phantasy Star",
  "Phantasy Star II",
  "Phantasy Star III: Generations of Doom",
  "Phantasy Star IV: The End of the Millennium",
  "Phantasy Star Online",
  "Phantasy Star Online 2: The Animation",
  "Phantasy Star Online Episode I & II",
  "Phantasy Star Online Version 2",
  "Phantasy Star Universe",
  "Pick a Star",
  "Plastic Galaxy: The Story of Star Wars Toys",
  "Pop Star",
  "Pop Star",
  "Pop Star",
  "Pop Star Puppy",
  "Pop Star on Ice",
  "Porn Star Zombies",
  "Porn Star: The Legend of Ron Jeremy",
  "Potato Star 2013QR3",
  "Prelude: Dog Star Man",
  "Prelude: Dog Star Man",
  "Project Runway: All-Star Challenge",
  "Project Runway: All-Star Challenge",
  "Pup Star",
  "Pup Star: Better 2Gether",
  "Pup Star: World Tour",
  "Puppy Star Christmas",
  "Quand je serai star",
  "Quentin Tarantino's Star Wars",
  "ROH: All Star Extravaganza",
  "Radio Star",
  "Rail of the Star",
  "Rainbow Brite and the Star Stealer",
  "Reunification: 25 Years After Star Trek - The Next Generation",
  "Rhinoskin: The Making of a Movie Star",
  "Rifftrax: The Star Wars Holiday Special",
  "Ringo Rocket Star and His Song for Yuri Gagarin",
  "Rising Star",
  "Rising Star",
  "Rising Star",
  "Rising Star",
  "Robert Mitchum: The Reluctant Star",
  "Robot Chicken: Star Wars",
  "Robot Chicken: Star Wars Episode II",
  "Robot Chicken: Star Wars III",
  "Rock Star",
  "Rock Star",
  "Rock Star 101",
  "Rock Star: INXS",
  "Rock Star: Supernova",
  "Rogue One: A Star Wars Story",
  "Rogue One: A Star Wars Story - World Premiere",
  "Rogue One: A Star Wars Toy Story",
  "STAR [Space Traveling Alien Reject]",
  "Saber Rider and the Star Sheriffs",
  "Sally Morgan: Star Psychic",
  "Saving Star Wars",
  "Schlag den Star",
  "Science of Star Wars",
  "Scout: A Star Wars Story",
  "Second Star to the Left",
  "Sesame Street: All-Star Alphabet",
  "Shadow Star Narutaru",
  "Shaq's All Star Comedy Roast 2",
  "Shaquille O'Neal Presents: All Star Comedy Jam - Live from Atlanta",
  "Shaquille O'Neal Presents: All Star Comedy Jam - Live from Las Vegas",
  "Shaquille O'Neal Presents: All Star Comedy Jam - Live from Orlando",
  "Shaquille O'Neal Presents: All Star Comedy Jam - Live from Orlando",
  "Shaquille O'Neal Presents: All-Star Comedy Jam - Live from Dallas",
  "Shaquille O'neal All-star Comedy Jam: Live from Sin City",
  "She Likes Red Star",
  "Shine, Shine, My Star",
  "Shooting Star",
  "Shuten Doji: The Star Hand Kid 2 - Demon Battle in the Firefly Field",
  "Shuten Doji: The Star Hand Kid 4 - End Game",
  "Shuten Doji: The Star Hand Kid Volume 3 - Time War",
  "Sinatra 100: An All-Star Grammy Concert",
  "Sinatra: Dark Star",
  "Smash Mouth: All Star",
  "Solo: A Star Wars Story",
  "Son of the Morning Star",
  "Soul Star",
  "Sparkling Red Star",
  "Special zu 'Star Trek XI'",
  "Stalked by a Reality Star",
  "Star",
  "Star",
  "Star",
  "Star",
  "Star",
  "Star",
  "Star",
  "Star 34",
  "Star 80",
  "Star Academy",
  "Star Academy",
  "Star Academy",
  "Star Appeal",
  "Star Ballz",
  "Star Blazers",
  "Star Blazers 2199",
  "Star Blazers 2199: Odyssey of the Celestial Ark",
  "Star Boys",
  "Star Camp",
  "Star Control",
  "Star Control 2",
  "Star Control 3",
  "Star Cops",
  "Star Crossed",
  "Star Crystal",
  "Star Driver",
  "Star Drunk",
  "Star Dust",
  "Star Fairies",
  "Star Falls",
  "Star Fish of 'Shark Tale'",
  "Star Force: Fugitive Alien II",
  "Star Fox",
  "Star Fox 64",
  "Star Fox 64 3D",
  "Star Fox Adventures",
  "Star Fox Command",
  "Star Fox Zero: The Battle Begins",
  "Star Fox: Assault",
  "Star Garden",
  "Star Hill",
  "Star Hunter",
  "Star Kid",
  "Star Knight",
  "Star Leaf",
  "Star Magic Presents",
  "Star Maidens",
  "Star Maps",
  "Star Men",
  "Star Nextdoor",
  "Star Night at the Cocoanut Grove",
  "Star Ocean",
  "Star Ocean EX",
  "Star Ocean: The Last Hope",
  "Star Ocean: The Second Story",
  "Star Ocean: Till the End of Time",
  "Star Odyssey",
  "Star Party",
  "Star Paws",
  "Star Pilot",
  "Star Pink",
  "Star Portal",
  "Star Portraits with Rolf Harris",
  "Star Power: The Creation of United Artists",
  "Star Quality",
  "Star Quality: Bon Voyage",
  "Star Quality: Mr. and Mrs. Edgehill",
  "Star Quest: The Odyssey",
  "Star Raiders",
  "Star Raiders: The Adventures of Saber Raine",
  "Star Reformer",
  "Star Reporter",
  "Star Runner",
  "Star Runners",
  "Star Search",
  "Star Shaped Scar",
  "Star Slammer",
  "Star Spangled Banners",
  "Star Spangled Girl",
  "Star Spangled Rhythm",
  "Star Spangled to Death",
  "Star Stories",
  "Star Street: The Adventures of the Star Kids",
  "Star Struck",
  "Star Struck",
  "Star Struck",
  "Star Struck",
  "Star Stuff: The Story of Carl Sagan",
  "Star Tech",
  "Star Time",
  "Star Tonight",
  "Star Tours",
  "Star Tours: The Adventures Continue",
  "Star Trak",
  "Star Trek",
  "Star Trek",
  "Star Trek",
  "Star Trek",
  "Star Trek",
  "Star Trek",
  "Star Trek 25th Anniversary Special",
  "Star Trek Adventure",
  "Star Trek Beyond",
  "Star Trek Continues",
  "Star Trek Dark Armada",
  "Star Trek Evolutions",
  "Star Trek I: Specter of the Past",
  "Star Trek II: The Wrath of Khan",
  "Star Trek III: The Search for Spock",
  "Star Trek IV: The Voyage Home",
  "Star Trek Insurrection Review",
  "Star Trek Into Darkness",
  "Star Trek Logs: An MTV Big Picture Special Edition",
  "Star Trek New Voyages: Phase II",
  "Star Trek Online",
  "Star Trek Secret Voyage: Whose Birth These Triumphs Are",
  "Star Trek V: The Final Frontier",
  "Star Trek V: The Final Frontier",
  "Star Trek VI: The Undiscovered Country",
  "Star Trek Versus Batman",
  "Star Trek Voyager: Elite Force",
  "Star Trek: 25th Anniversary Enhanced",
  "Star Trek: 30 Years and Beyond",
  "Star Trek: A Captain's Log",
  "Star Trek: A New Vision",
  "Star Trek: Aliens",
  "Star Trek: Armada",
  "Star Trek: Armada II",
  "Star Trek: Away Team",
  "Star Trek: Beyond the Final Frontier",
  "Star Trek: Birth of the Federation",
  "Star Trek: Borg",
  "Star Trek: Bridge Commander",
  "Star Trek: Bridge Crew",
  "Star Trek: Captain Pike",
  "Star Trek: Chains of Betrayal",
  "Star Trek: Deception",
  "Star Trek: Deep Space Nine",
  "Star Trek: Deep Space Nine - Dominion Wars",
  "Star Trek: Deep Space Nine - Harbinger",
  "Star Trek: Deep Space Nine - The Fallen",
  "Star Trek: Deep Space Nine Companion",
  "Star Trek: Disco Generation",
  "Star Trek: Discovery",
  "Star Trek: D·A·C",
  "Star Trek: Elite Force II",
  "Star Trek: Encounters",
  "Star Trek: Enterprise",
  "Star Trek: Enterprise - In Conversation - The First Crew",
  "Star Trek: Enterprise - In a Time of War",
  "Star Trek: Enterprise - Uncharted Territory",
  "Star Trek: First Contact",
  "Star Trek: First Contact Review",
  "Star Trek: GENESIS",
  "Star Trek: Gene Rodenberry's Vision",
  "Star Trek: Generations",
  "Star Trek: Generations",
  "Star Trek: Hidden Evil",
  "Star Trek: Hidden Frontier",
  "Star Trek: Horizon",
  "Star Trek: Inside the Roddenberry Vault",
  "Star Trek: Insurrection",
  "Star Trek: Intrepid",
  "Star Trek: Invasion",
  "Star Trek: Invincible",
  "Star Trek: Judgment Rites",
  "Star Trek: Klingon",
  "Star Trek: Legacy",
  "Star Trek: Nemesis",
  "Star Trek: New Worlds",
  "Star Trek: Odyssey",
  "Star Trek: Of Gods and Men",
  "Star Trek: Operation Beta Shield",
  "Star Trek: Osiris",
  "Star Trek: Phoenix - Cloak & Dagger Part I",
  "Star Trek: Phoenix - No Other Medicine",
  "Star Trek: Picard",
  "Star Trek: Pinball",
  "Star Trek: Progeny",
  "Star Trek: Renegades",
  "Star Trek: Science vs. Fiction",
  "Star Trek: Score",
  "Star Trek: Secrets of the Universe",
  "Star Trek: Shattered Universe",
  "Star Trek: Short Treks",
  "Star Trek: Starfleet Academy",
  "Star Trek: Starfleet Command",
  "Star Trek: Starfleet Command III",
  "Star Trek: Starfleet Command: Orion Pirates",
  "Star Trek: Starfleet Command: Volume II: Empires at War",
  "Star Trek: Starships",
  "Star Trek: Strategic Operations Simulator",
  "Star Trek: Tactical Assault",
  "Star Trek: The Animated Series",
  "Star Trek: The Experience - Borg Invasion 4D",
  "Star Trek: The Experience - The Klingon Encounter",
  "Star Trek: The Gag Reel",
  "Star Trek: The Game Show",
  "Star Trek: The Next Generation",
  "Star Trek: The Next Generation - A Final Unity",
  "Star Trek: The Next Generation - Regeneration: Engaging the Borg",
  "Star Trek: The Next Generation - Survive and Suceed: An Empire at War",
  "Star Trek: The Next Generation Companion",
  "Star Trek: The Next Generation Interactive Technical Manual",
  "Star Trek: The Next Generation: Interactive VCR Board Game - A Klingon Challenge",
  "Star Trek: The Original Series",
  "Star Trek: The Pepsi Generation",
  "Star Trek: The Rebel Universe",
  "Star Trek: To Boldly Go",
  "Star Trek: Voyager",
  "Star Troopers",
  "Star Vehicle",
  "Star Wait",
  "Star Warp'd",
  "Star Warriors",
  "Star Wars",
  "Star Wars",
  "Star Wars 1313",
  "Star Wars Battlefront: Renegade Squadron",
  "Star Wars Begins: A Filmumentary",
  "Star Wars Blips",
  "Star Wars Cantina Karaoke",
  "Star Wars Celebration 2017",
  "Star Wars City",
  "Star Wars Downunder",
  "Star Wars Droids: The Jawa Adventure",
  "Star Wars Elevator Prank",
  "Star Wars Empire at War: Forces of Corruption",
  "Star Wars Episode V 1/2: The Han Solo Affair",
  "Star Wars Forces of Destiny: Volume 2",
  "Star Wars Forces of Destiny: Volume 3",
  "Star Wars Galaxies: An Empire Divided",
  "Star Wars Galaxies: Jump to Lightspeed",
  "Star Wars Galaxies: Rage of the Wookiees",
  "Star Wars Galaxies: The Total Experience",
  "Star Wars Galaxy of Adventures",
  "Star Wars Galaxy's Edge: Adventure Awaits",
  "Star Wars Gangsta Rap",
  "Star Wars Heroes & Villains",
  "Star Wars Jedi: Fallen Order",
  "Star Wars Legends: Legacy of the Force",
  "Star Wars Pathways: Chapter I - Path of Betrayal",
  "Star Wars Rebels",
  "Star Wars Resistance",
  "Star Wars SC 38 Reimagined",
  "Star Wars Spoofs",
  "Star Wars Tech",
  "Star Wars Uncut: Director's Cut",
  "Star Wars: A Galaxy in Darkness",
  "Star Wars: A Musical Journey",
  "Star Wars: Aeon",
  "Star Wars: Battlefront",
  "Star Wars: Battlefront",
  "Star Wars: Battlefront II",
  "Star Wars: Bounty Hunter",
  "Star Wars: Clone Wars",
  "Star Wars: Connections",
  "Star Wars: Dark Forces",
  "Star Wars: Demolition",
  "Star Wars: Destroyer",
  "Star Wars: Dresca",
  "Star Wars: Droids",
  "Star Wars: Empire at War",
  "Star Wars: Episode I - Battle for Naboo",
  "Star Wars: Episode I - Jedi Power Battles",
  "Star Wars: Episode I - Racer",
  "Star Wars: Episode I - The Gungan Frontier",
  "Star Wars: Episode I - The Phantom Menace",
  "Star Wars: Episode I - The Phantom Menace",
  "Star Wars: Episode II - Attack of the Clones",
  "Star Wars: Episode III - Revenge of the Sith",
  "Star Wars: Episode III - Revenge of the Sith",
  "Star Wars: Episode IV - A New Hope",
  "Star Wars: Episode IV - A Toy Story",
  "Star Wars: Episode IV: A New Hope - Deleted Scenes",
  "Star Wars: Episode IX - The Rise of Skywalker",
  "Star Wars: Episode V - The Empire Strikes Back",
  "Star Wars: Episode V - The Empire Strikes Back: Deleted Scenes",
  "Star Wars: Episode VI - Return of the Jedi",
  "Star Wars: Episode VII - The Force Awakens",
  "Star Wars: Episode VII - The Force Awakens: The Story Awakens - The Table Read",
  "Star Wars: Episode VIII - The Last Jedi",
  "Star Wars: Episode XXXVIII - 10 Years Later",
  "Star Wars: Evolution of the Lightsaber Duel",
  "Star Wars: Extintion",
  "Star Wars: Force Commander",
  "Star Wars: Forces of Destiny",
  "Star Wars: Galactic Battlegrounds",
  "Star Wars: Galaxies - Trials of Obi-Wan",
  "Star Wars: Generations",
  "Star Wars: Greatest Moments",
  "Star Wars: Hoshino",
  "Star Wars: Jedi Arena",
  "Star Wars: Jedi Knight - Dark Forces II",
  "Star Wars: Jedi Knight - Jedi Academy",
  "Star Wars: Jedi Knight - Mysteries of the Sith",
  "Star Wars: Jedi Knight II - Jedi Outcast",
  "Star Wars: Jedi Starfighter",
  "Star Wars: Knights of the Old Republic",
  "Star Wars: Knights of the Old Republic II - The Sith Lords",
  "Star Wars: Launch Bay - Meet the Makers",
  "Star Wars: Masters of Teräs Käsi",
  "Star Wars: Music by John Williams",
  "Star Wars: Obi-Wan",
  "Star Wars: Qui-Gon Jinn",
  "Star Wars: Qui-Gon Jinn II",
  "Star Wars: Qui-Gon Jinn III",
  "Star Wars: Racer Revenge",
  "Star Wars: Rebel Assault",
  "Star Wars: Rebel Assault II - the Hidden Empire",
  "Star Wars: Rebellion",
  "Star Wars: Republic Commando",
  "Star Wars: Return of the Jedi - Death Star Battle",
  "Star Wars: Revelations",
  "Star Wars: Rise of the Resistance",
  "Star Wars: Rogue Squadron",
  "Star Wars: Rogue Squadron II - Rogue Leader",
  "Star Wars: Rogue Squadron III - Rebel Strike",
  "Star Wars: Shadows of the Empire",
  "Star Wars: Shortened!",
  "Star Wars: Star Warriors",
  "Star Wars: Star Warriors",
  "Star Wars: Starfighter",
  "Star Wars: Super Bombad Racing",
  "Star Wars: TIE Fighter",
  "Star Wars: The Apprentice",
  "Star Wars: The Clone Wars",
  "Star Wars: The Clone Wars",
  "Star Wars: The Clone Wars",
  "Star Wars: The Clone Wars",
  "Star Wars: The Clone Wars - Republic Heroes",
  "Star Wars: The Clone Wars Preview Special",
  "Star Wars: The Empire Strikes Back",
  "Star Wars: The Empire Strikes Back",
  "Star Wars: The Force Unleashed",
  "Star Wars: The Force Unleashed",
  "Star Wars: The Force Unleashed - Ultimate Sith Edition",
  "Star Wars: The Force Unleashed II",
  "Star Wars: The Force Unleashed II",
  "Star Wars: The Force and the Fury",
  "Star Wars: The Last Jedi",
  "Star Wars: The Last Jedi Cast Live Q&A",
  "Star Wars: The Legacy Revealed",
  "Star Wars: The Lesser Evil",
  "Star Wars: The Magic & the Mystery",
  "Star Wars: The New Republic Anthology",
  "Star Wars: The Old Republic",
  "Star Wars: The Old Republic - Knights of the Fallen Empire",
  "Star Wars: The Old Republic - Rise of the Hutt Cartel",
  "Star Wars: The Old Republic - Shadow of Revan",
  "Star Wars: The Scarlet Lance",
  "Star Wars: Threads of Destiny",
  "Star Wars: Wrath of the Mandalorian",
  "Star Wars: X-Wing",
  "Star Wars: X-Wing Alliance",
  "Star Wars: X-Wing vs. TIE Fighter",
  "Star Wars: Yoda Stories",
  "Star Watching Dog",
  "Star Whackers",
  "Star Without Light",
  "Star Witness",
  "Star Wolf",
  "Star Worms II: Attack of the Pleasure Pods",
  "Star Wreck",
  "Star Wreck 2: The Old Shit",
  "Star Wreck 2pi: Full Twist, Now!",
  "Star Wreck 3",
  "Star Wreck IV: The Kilpailu",
  "Star Wreck: In the Pirkinning",
  "Star Wreck: Lost Contact",
  "Star académie",
  "Star in the Dust",
  "Star in the Night",
  "Star na si Van Damme Stallone",
  "Star of India",
  "Star of Jaipur",
  "Star of Midnight",
  "Star of My Night",
  "Star of the Family",
  "Star suburb: La banlieue des étoiles",
  "Star vs. the Forces of Evil",
  "Star!",
  "Star-Crossed",
  "Star-Crossed",
  "Star-Studded Spoof of the New TV Season, G-Rated, with Glamour, Glitter and Gags",
  "Star-ving",
  "Stardate Revisited: The Origin of Star Trek - The Next Generation",
  "Starmania. Österreich sucht den neuen Star",
  "Sternberg - Shooting Star",
  "Stevie Wonder Songs in the Key of Life an All Star Grammy Salute",
  "Still Star-Crossed",
  "Style Star",
  "Sun, Moon and Star: Part 2",
  "Super Star",
  "Super Star",
  "Super Star Wars",
  "Super Star Wars: Return of the Jedi",
  "Super Star Wars: The Empire Strikes Back",
  "Tar with a Star",
  "Teen Star Academy",
  "Tell It to a Star",
  "Texaco Star Theater: Opening Night",
  "Texaco Star Theatre",
  "That movie is not Star wars. Episode VII. Last jedi",
  "The All Star Impressions Show",
  "The All-Star Bond Rally",
  "The Ancient Magus' Bride: Those Awaiting a Star Part 2",
  "The Art Star and the Sudanese Twins",
  "The Beautiful World of Jeffree Star",
  "The Belle Star Story",
  "The Blue Star Hotel",
  "The Brightest Star in the Sky",
  "The Broken Star",
  "The Buggles: Video Killed the Radio Star",
  "The Butcher, the Star and the Orphan",
  "The Car That Became a Star",
  "The Car's the Star",
  "The Characters of 'Star Wars'",
  "The Child Star Jinx",
  "The Christmas Star",
  "The Cloud-Capped Star",
  "The Cowboy Star",
  "The Cowboy and the Movie Star",
  "The Dark Star",
  "The Day I Bought a Star",
  "The Evening Star",
  "The Falling Star",
  "The Falling Star",
  "The Force Is with Them: The Legacy of 'Star Wars'",
  "The Four Star Boarder",
  "The Hypothetical Star Wars Holiday Special",
  "The Last Movie Star",
  "The Lone Star Kid",
  "The Lone Star Ranger",
  "The Lone Star State",
  "The Lone Star Trail",
  "The Lucky Star",
  "The Making of 'Star Trek: First Contact'",
  "The Making of 'Star Wars'",
  "The Making of a Rock Star",
  "The Missing Star",
  "The Muppets All-Star Comedy Gala",
  "The Mythology of 'Star Wars'",
  "The Next Star",
  "The North Star",
  "The North Star",
  "The North Star",
  "The Official Star Wars Fan Film Awards",
  "The One: Making a Music Star",
  "The Poker Star",
  "The Powers of Matthew Star",
  "The Prince and the Evening Star",
  "The Princess with the Golden Star",
  "The Raccoons and the Lost Star",
  "The Real Death Star",
  "The Redd Foxx Becomes a Movie Star",
  "The Romancing Star",
  "The Science of Star Trek",
  "The Secret World of Jeffree Star",
  "The Shining Star of Losers Everywhere",
  "The Shooting Star Salesman",
  "The Silver Star",
  "The Soap-Suds Star",
  "The Soldier's Star",
  "The Southern Star",
  "The Southern Star",
  "The Star",
  "The Star",
  "The Star",
  "The Star",
  "The Star Boarder",
  "The Star Chamber",
  "The Star Dreamer",
  "The Star Inspector",
  "The Star Jones Reynolds Report",
  "The Star Maker",
  "The Star Maker",
  "The Star Maker",
  "The Star Packer",
  "The Star Trek Encyclopedia",
  "The Star Wagon",
  "The Star Wars Holiday Special",
  "The Star Wars Show",
  "The Star Witness",
  "The Star and the Sea",
  "The Star and the Story",
  "The Star of Bethlehem",
  "The Star of Bethlehem",
  "The Star of Bethlehem",
  "The Star of Bethlehem",
  "The Star of Christmas",
  "The Star of Cottonland",
  "The Stars of 'Star Wars': Interviews from the Cast",
  "The State's 43rd Annual All-Star Halloween Special",
  "The Stories: The Making of 'Rogue One: A Star Wars Story'",
  "The Story of Star Wars",
  "The Sun Is Also a Star",
  "The Tin Star",
  "The Unauthorized 'Star Wars' Story",
  "The Universe's Star",
  "The Untitled Star Wars Mockumentary",
  "The Whispering Star",
  "The Wormwood Star",
  "The Yellow Star: The Persecution of the Jews in Europe - 1933-1945",
  "The Young Comedians All-Star Reunion",
  "The Young Person's Guide to Becoming a Rock Star",
  "Third Star",
  "Tin Star",
  "To Be a Star",
  "Tony Blair: Rock Star",
  "Top Star",
  "Top Star Yoo Baek",
  "Top star magazín",
  "Tri-Star",
  "Twenty Dollar Star",
  "Twin Star Exorcists",
  "Twinkle Twinkle Little Star",
  "Ultimate Trek: Star Trek's Greatest Moments",
  "Ultraman Tiga & Ultraman Dyna: Warriors of the Star of Light",
  "Under the North Star",
  "Under the North Star II",
  "Unser Star für Oslo",
  "Video Killed the Radio Star",
  "Voices of a Distant Star",
  "WWE: John Morrison - Rock Star",
  "WWF All-Star Wrestling",
  "What We Left Behind: Star Trek DS9",
  "When Star Wars Ruled the World",
  "White Star",
  "Why Can't I Be a Movie Star?",
  "William Shatner's Star Trek Memories",
  "Wish Upon a Star",
  "Wishes on a Falling Star",
  "Working Class Rock Star",
  "World Premiere 'Star Wars III: Revenge of the Sith'",
  "Wyclef Jean: All Star Jam at Carnegie Hall",
  "Yogi Bear's All-Star Comedy Christmas Caper",
  "You're a Star",
  "À la recherche de la nouvelle star",
  "È nata una star?"
]

In [42]:
intent_dataset = {
    "get_film_duration": [
        "Сколько идет фильм",
        "Длительность фильма",
        "Продолжительность фильма",
        "Какая длительность фильма",
        "Какая продолжительность у фильма",
        "Сколько часов идет фильм",
        "Сколько длится фильм",
        "Какова длительность фильма",
        "Сколько времени идет фильм",
        "Сколько длится этот фильм",
        "Сколько времени продолжается фильм",
        "Как долго идет фильм",
        "Как долго длится фильм",
        "Какова продолжительность фильма",
        "Сколько времени фильм идет",
        "Сколько времени длится фильм",
        "На сколько времени растянут фильм",
        "Сколько минут идет фильм",
        "Какая продолжительность у фильма",
        "Длина фильма",
    ],
    "get_film_genre": [
        "Какой жанр у фильма",
        "Что за жанр у фильма",
        "Жанр фильма",
        "К чему относится фильм",
        "Какой жанр фильма",
        "Тип фильма",
        "Жанры фильма",
        "Какой стиль у фильма",
        "Какой тип у фильма",
        "Что за стиль у фильма",
        "Какие жанры у фильма",
        "Какой жанр картины",
        "Какой жанровый ряд у фильма",
        "Какой жанр этого фильма",
        "Что за жанровая категория у фильма",
        "Какую категорию жанра имеет фильм",
        "В каком жанре этот фильм",
        "Какой фильм по жанру",
        "В какой жанр входит фильм",
        "Какой жанровый стиль у фильма",
        "Что за категория фильма",
        "Что за жанр у картины",
        "Какова жанровая принадлежность фильма",
        "Жанровая категория фильма",
        "Какой тип картины",
        "Что за тип у фильма",
        "Что это за жанр у фильма",
        "Какая категория у фильма",
        "Какая жанровая категория у фильма",
        "Какого жанра фильм",
        "Какая жанровая спецификация у фильма"
    ],
    "get_film_rating": [
        "Рейтинг",
        "Рейтинг фильма",
        "Какой рейтинг у фильма",
        "Какая оценка у фильма",
        "Оценка фильма",
        "Какой рейтинг фильма",
        "Какую оценку получил фильм",
        "Рейтинг фильма",
        "Оценка фильма",
        "Какой балл у фильма",
        "Какой рейтинг у этого фильма",
        "Какая оценка у этого фильма",
        "Что за рейтинг у фильма",
        "Оценка этого фильма"
    ],
    "get_film_actors": [
        "Кто снимался в фильме",
        "Актеры фильма",
        "Актеры в фильме",
        "Кто участвовал в фильме",
        "Кто играл в фильме",
        "Какие актеры снимались в фильме",
        "Кто сыграл в фильме",
        "Состав актеров фильма",
        "Кто играл в фильме",
        "Актерский состав фильма",
        "Кто был в ролях фильма",
        "Какие актеры участвовали в фильме",
        "Кто играл в главных ролях",
        "Актеры, снимавшиеся в фильме",
        "Кто был в фильме",
        "В каких фильмах играл актер",
        "Фильмы, где участвовал актер",
        "В каких проектах снимался актер",
        "Фильмы с участием актера",
        "Какие фильмы с этим актером",
        "Фильмы, где играл актер",
        "Какие роли у актера в кино",
        "Кинопроекты актера",
        "Где этот актер снимался",
        "Фильмы, в которых актер участвовал",
        "Фильмы, где актер играл роли",
        "В каких фильмах участвовал актер",
        "Фильмы, где появлялся актер",
        "В каких фильмах снимался актер",
        "Где снимался актер",
        "Где снимался",
        "Фильмы с актером",
        "Фильмы с"
    ],
    "get_film_directors": [
        "Кто режиссер фильма",
        "Режиссер фильма",
        "Кто снимал",
        "Кто создал фильм",
        "Кто поставил фильм",
        "Кто режиссировал фильм",
        "Кто снял фильм",
        "Чей фильм",
        "Кто создатель фильма",
        "Кто работал режиссером фильма",
        "Кто автор фильма",
        "Кто был режиссером фильма",
        "Режиссер какой у фильма",
        "Кто руководил съемками фильма",
        "Какие фильмы создал режиссер",
        "Фильмы, снятые этим режиссером",
        "Какие фильмы он снял",
        "Работы режиссера",
        "Фильмы этого режиссера",
        "Что снял режиссер",
        "Какие фильмы режиссировал",
        "Что создал режиссер",
        "Фильмы, сделанные этим режиссером",
        "Проекты режиссера в кино",
        "Какие фильмы поставил режиссер",
        "Фильмы, в которых работал режиссер",
        "Работы этого режиссера в кино",
        "Какие фильмы снял режиссер",
        "Фильмы от режиссера",
    ],
    "get_films_by_genre": [
        "Какие фильмы есть в жанре",
        "Фильмы в жанре",
        "Фильмы по жанру",
        "Список фильмов в жанре",
        "Что посмотреть в жанре",
        "Фильмы, относящиеся к жанру",
        "Какие фильмы подходят под жанр",
        "Фильмы категории",
        "Что за фильмы в жанре",
        "Что есть в жанре",
        "Кино в жанре",
        "Фильмы какого жанра",
        "Фильмы жанра",
        "Какие фильмы в жанре",
        "Какой жанр у фильмов",
        "Что показать в жанре",
        "Какие картины в жанре",
        "Что посмотреть по жанру",
        "Фильмы с жанром",
        "Жанровые фильмы",
        "Какие картины по жанру",
        "Фильмы по категории",
        "Что в жанре",
        "Фильмы в этой категории",
        "Какие фильмы в этом жанре",
        "По жанру фильмов"
    ],
    "get_film_writers": [
        "Кто написал сценарий к фильму",
        "Кто написал фильм",
        "Кто написал сценарий фильма",
        "Кто написал сценарий к фильму",
        "Автор сценария фильма",
        "Сценарист фильма",
        "Кто является сценаристом фильма",
        "Кто создал сценарий фильма",
        "Кто работал над сценарием фильма",
        "Сценарий фильма написал кто",
        "Кто автор текста фильма",
        "Кто ответственен за сценарий фильма",
        "Сценарист какой у фильма",
        "Кто написал текст для фильма",
        "Чей сценарий у фильма",
        "Кто писал сюжет фильма",
        "Кто разработал сценарий фильма",
        "Какие фильмы написал этот сценарист",
        "Фильмы, которые написал сценарист",
        "Какие фильмы он написал",
        "Сценарист каких фильмов",
        "Что написал сценарист",
        "Какие фильмы написаны этим сценаристом",
        "Фильмы, написанные этим сценаристом",
        "Что сценарист создал",
        "Какие фильмы написал автор сценария",
        "Фильмы с участием этого сценариста",
        "Работы сценариста в кино",
        "Что из фильмов написал сценарист",
        "Какие фильмы он создавал как сценарист"
    ],
    "get_film_description": [
        "О чем фильм",
        "Какой сюжет у фильма",
        "О фильме",
        "Какой сюжет фильма",
        "О чем",
        "Что за сюжет у фильма",
        "Что за сюжет фильма",
        "Опишите фильм",
        "Что рассказывает фильм",
        "О чем сюжет фильма",
        "Дайте описание фильма",
        "Каков сюжет фильма",
        "Фильм про что",
        "Какой рассказ у фильма",
        "Сюжет фильма в двух словах",
        "Кратко о фильме",
        "Что происходит в фильме",
        "Что за история в фильме",
        "Фильм о чем",
        "Какая идея фильма",
        "Какой рассказ в фильме",
        "Как фильм рассказывает историю",
        "Что в фильме",
        "Сюжет в нескольких словах",
        "Что показывает фильм",
        "Что ожидает зритель в фильме",
        "Каковы события фильма",
        "Что происходило в фильме",
        "Какое содержание фильма",
        "Какова основная идея фильма",
        "Что мы видим в фильме",
        "Какие события в фильме",
        "О чём сюжет картины",
        "Про что фильм",
        "Что раскрывает фильм",
        "Какова концепция фильма"
    ],
    "get_similar_films": [
        "Похожие фильмы на",
        "Какие фильмы похожи на",
        "Предложи похожие фильмы на",
        "Похожее на",
        "Схожее с",
        "Что-то похожее на",
        "По типу",
        "Фильмы, похожие на",
        "Покажи фильмы, похожие на",
        "Какие еще фильмы похожи на",
        "Подобные фильмы на",
        "Фильмы аналогичные",
        "Советуйте похожие фильмы на",
        "Список фильмов, похожих на",
        "Фильмы схожие с",
        "Что похоже на",
        "Фильмы, которые напоминают",
        "Кино, похожее на",
        "Фильмы в стиле",
        "Рекомендации по фильмам, похожим на",
    ],
}

### Перевод

In [46]:
model_name = 'Helsinki-NLP/opus-mt-ru-en'
model = MarianMTModel.from_pretrained(model_name)
tokenizer = MarianTokenizer.from_pretrained(model_name)

def translate(text: str) -> str:
    inputs = tokenizer(text, return_tensors="pt", padding=True)

    translated = model.generate(**inputs)
    
    return tokenizer.decode(translated[0], skip_special_tokens=True)

In [47]:
text = "покажи мне фильмы от Алана Бэкера. Люблю жанр приключения. Еще нравятся звездные войны"
translated_text = translate(text)
print(translated_text)

Show me the movies from Alan Backer, I love adventure genre, I also like star wars.


## 1. Classify User Intention

In [43]:
intent_dataset_array = []

for intent, examples in intent_dataset.items():
    for example in examples:
        intent_dataset_array.append({"intent": intent, "sentence": example})

df = pd.DataFrame(intent_dataset_array)

print(df.head())

              intent                          sentence
0  get_film_duration                Сколько идет фильм
1  get_film_duration               Длительность фильма
2  get_film_duration          Продолжительность фильма
3  get_film_duration         Какая длительность фильма
4  get_film_duration  Какая продолжительность у фильма


In [44]:
X = df['sentence']
y = df['intent']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

vectorizer = TfidfVectorizer()
X_train_tfidf = vectorizer.fit_transform(X_train)
X_test_tfidf = vectorizer.transform(X_test)

model = LogisticRegression()
model.fit(X_train_tfidf, y_train)

y_pred = model.predict(X_test_tfidf)

print(classification_report(y_test, y_pred))

new_text = "покажи мне фильмы от Алана Бэкера. Люблю жанр приключения. Еще нравятся звездные войны"
new_text_tfidf = vectorizer.transform([new_text])
predicted_intent = model.predict(new_text_tfidf)[0]

print(f"Предсказанный интент: {predicted_intent}")

                      precision    recall  f1-score   support

     get_film_actors       0.71      1.00      0.83         5
get_film_description       1.00      0.73      0.84        11
  get_film_directors       0.62      1.00      0.77         5
      get_film_genre       0.86      1.00      0.92         6
     get_film_rating       1.00      1.00      1.00         1
    get_film_writers       1.00      0.67      0.80         9
  get_films_by_genre       1.00      1.00      1.00         4
   get_similar_films       1.00      1.00      1.00         5

            accuracy                           0.87        46
           macro avg       0.90      0.92      0.90        46
        weighted avg       0.91      0.87      0.87        46

Предсказанный интент: get_film_genre


In [145]:
joblib.dump(vectorizer, "./services/assistant/models/vectorizer.pkl")
joblib.dump(model, "./services/assistant/models/intent_model.pkl")

['./services/assistant/models/intent_model.pkl']

## 2. Entity Extracting

**Люди**

In [ ]:
tagger = SequenceTagger.load("ner")

In [15]:
transcript = translated_text

sentence = Sentence(transcript)
tagger.predict(sentence)

persons = [entity.text for entity in sentence.get_spans('ner') if entity.get_label("ner").value == "PER"]
print(persons)

['Alan Backer']


**Жанры**

In [20]:
transcript = translated_text

genres = []
for genre in genres_titles:
    pattern = re.compile(rf"\b{re.escape(genre.lower())}\b", re.IGNORECASE)
    if pattern.search(transcript):
        genres.append(genre)

print(genres)

['Adventure']


**Фильмы**

In [21]:
transcript = translated_text

films = []

for film in films_titles:
    pattern = re.compile(rf"\b{re.escape(film.lower())}\b", re.IGNORECASE)
    matches = pattern.findall(transcript)
    
    if matches and film not in films:
        films.append(film)

print(films)

['Star', 'Star Wars']
